In [1]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, evaluation
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SequentialEvaluator
from sentence_transformers.readers import STSBenchmarkDataReader
import logging
import sys
import os
import torch
import numpy as np
import zipfile
import io
import pandas as pd

In [5]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
# Load a named sentence model (based on BERT). This will download the model from our server.
# Alternatively, you can also pass a filepath to SentenceTransformer()
model = SentenceTransformer(model_name)

2023-12-21 09:49:18 - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
2023-12-21 09:49:24 - Use pytorch device: cuda


In [6]:
logging.info("Read STS test dataset")
filepath = '../stsbenchmark/sts-test_th.csv'
thai_sts = pd.read_csv(filepath,header=None).dropna()
thai_sts.columns = ['text_1','text_2','correlation']

2023-12-21 09:49:24 - Read STS test dataset


In [7]:
thai_sts

,text_1,text_2,correlation
0,ผู้หญิงกำลังจัดแต่งทรงผมของเธอ,ผู้หญิงคนหนึ่งกำลังแปรงผมของเธอ,2.5
1,กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด,กลุ่มเด็กผู้ชายกำลังเล่นฟุตบอลบนชายหาด,3.6
2,ผู้หญิงคนหนึ่งกำลังวัดข้อเท้าของผู้หญิงคนอื่น,ผู้หญิงวัดข้อเท้าของผู้หญิงคนอื่น,5.0
3,ผู้ชายกำลังตัดแตงกวา,ผู้ชายกำลังหั่นแตงกวา,4.2
4,ผู้ชายกำลังเล่นพิณ,ผู้ชายกำลังเล่นแป้นพิมพ์,1.5
...,...,...,...
1374,ฟิลิปปินส์แคนาดาให้คำมั่นว่าจะเพิ่มความสัมพันธ...,ฟิลิปปินส์ช่วยประหยัด 100 หลังจมูก,0.0
1375,ชาวอิสราเอลบาร์ชาวปาเลสไตน์จากเมืองเก่าของเยรู...,การแก้ปัญหาสองรัฐระหว่างชาวปาเลสไตน์อิสราเอลพา...,1.0
1376,คุณรู้เกี่ยวกับหน่วยสืบราชการลับมากแค่ไหน?,ผู้ร่างกฎหมายจากทั้งสองฝ่ายแสดงความไม่พอใจที่ห...,1.0
1377,โอบามาดิ้นรนเพื่อบรรเทาความกลัวของซาอุดิอาระเบ...,พม่าพยายามดิ้นรนเพื่อสรุปรายชื่อผู้มีสิทธิเลือ...,0.0


In [8]:
test_evaluator = EmbeddingSimilarityEvaluator(thai_sts['text_1'], thai_sts['text_2'], thai_sts['correlation'], batch_size=16, show_progress_bar=False)
test_evaluator(model) #ใช้แค่ cosine-similarity: spearman: XXXXX

2023-12-21 09:49:24 - EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:
2023-12-21 09:49:28 - Cosine-Similarity :	Pearson: 0.7925	Spearman: 0.8049
2023-12-21 09:49:28 - Manhattan-Distance:	Pearson: 0.7940	Spearman: 0.7965
2023-12-21 09:49:28 - Euclidean-Distance:	Pearson: 0.7954	Spearman: 0.7984
2023-12-21 09:49:28 - Dot-Product-Similarity:	Pearson: 0.7183	Spearman: 0.7057


0.8049417898176724